In [1]:
from torch.utils.data import Dataset, DataLoader

In [2]:
import MLSE_Class as MLSE

In [3]:
X_train_eclipse,Y_train_bugs_eclipse,X_test_eclipse,Y_test_bugs_eclipse = MLSE.classification_binary("differentSoftware/eclipse/eclipse_all_data.csv",
                                                                     'numberOfBugsFoundUntil',"wmc",0,
                                                                       "bugs")

AlltheMatrix/differentSoftware/eclipse/eclipse_all_data.csv
lens: 50
start:2,end:43
(997,)
the data shape is (997, 40)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[0 0 0 ... 0 2 17]
 [0 0 0 ... 0 6 16]
 [2 0 0 ... 0 3 13]
 ...
 [6 1 1 ... 0 33 2062]
 [14 3 1 ... 0 10 79]
 [10 0 0 ... 0 7 61]] (698,)
@@@@@@@@@@@@@@@@@@@@@@@@@@


In [6]:
from imblearn.combine import SMOTEENN
from collections import Counter
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X_train_eclipse, Y_train_bugs_eclipse)
print(f'Y_resampled: {sorted(Counter(y_resampled).items())}')

Y_resampled: [(False, 388), (True, 470)]


In [7]:
class ImageDataset(Dataset):
    def __init__(self, file_list, target_list):
        self.file_list = file_list
        self.target_list = target_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img = self.file_list[index]
        label = self.target_list[index]
        return img, label

In [10]:
import torch
import numpy as np

In [12]:
X_train_eclipse_Y = torch.from_numpy(np.array(X_resampled, dtype=np.float32))
Y_train_bugs_eclipse_Y = torch.from_numpy(np.array(y_resampled, dtype=np.float32))
trainset_Y = ImageDataset(X_train_eclipse_Y, Y_train_bugs_eclipse_Y)

In [13]:
X_train_eclipse = torch.from_numpy(np.array(X_train_eclipse, dtype=np.float32))
Y_train_bugs_eclipse = torch.from_numpy(np.array(Y_train_bugs_eclipse, dtype=np.float32))
trainset = ImageDataset(X_train_eclipse, Y_train_bugs_eclipse)

In [14]:
X_test_eclipse = torch.from_numpy(np.array(X_test_eclipse, dtype=np.float32))
Y_test_bugs_eclipse = torch.from_numpy(np.array(Y_test_bugs_eclipse, dtype=np.float32))
testset = ImageDataset(X_test_eclipse, Y_test_bugs_eclipse)

In [15]:
len(trainset.target_list)

698

In [71]:
len(X_train_eclipse)

698

In [16]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets, models
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
from collections import namedtuple
from IPython.display import Image
from torch.utils import data as Data
import torch.optim as optim
import copy
import os

In [17]:
train_dataloader = DataLoader(trainset, batch_size=64, shuffle=True, drop_last=False)
train_dataloader_Y = DataLoader(trainset_Y, batch_size=64, shuffle=True, drop_last=False)

In [18]:
test_dataloader = DataLoader(testset, batch_size=64, shuffle=True, drop_last=False)

In [19]:
class Simple_MLP(nn.Module):
    def __init__(self, size_list):
        super(Simple_MLP, self).__init__()
        layers = []
        self.size_list = size_list
        for i in range(len(size_list) - 2):
            layers.append(nn.Linear(size_list[i],size_list[i+1]))
            layers.append(nn.BatchNorm1d(num_features = size_list[i+1]))
            layers.append(nn.ReLU())
#             layers.append(nn.Dropout(0.2))
        layers.append(nn.Linear(size_list[-2], size_list[-1]))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [20]:
cuda = torch.cuda.is_available()
print(cuda)

model = Simple_MLP([40,80,160, 16, 2])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)
device = torch.device("cuda" if cuda else "cpu")
print(model)

False
Simple_MLP(
  (net): Sequential(
    (0): Linear(in_features=40, out_features=80, bias=True)
    (1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=80, out_features=160, bias=True)
    (4): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=160, out_features=16, bias=True)
    (7): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=16, out_features=2, bias=True)
  )
)


In [21]:
def train_epoch(model, train_loader, criterion, optimizer):
    scheduler.step()
    model.train()
    model.to(device)

    running_loss = 0.0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device).float()
        target = target.to(device).long() # all data & model on same device

        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    end_time = time.time()
    
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    return running_loss

In [35]:
from sklearn.metrics import recall_score  

In [36]:
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()
        model.to(device)

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0
        recall = []
        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.to(device).float()
            target = target.to(device).long()

            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()
            recall.append(recall_score(target, predicted))
            loss = criterion(outputs, target).detach()
            running_loss += loss.item()


        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Testing Loss: ', running_loss)
        print('Testing Accuracy: ', acc, '%')
        print('Testing Recall: ', np.mean(recall)*100, '%')
        return running_loss, acc

In [37]:
cuda = torch.cuda.is_available()
print(cuda)

model = Simple_MLP([40,80,160,160,640,20,10, 2])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)
device = torch.device("cuda" if cuda else "cpu")
print(model)

False
Simple_MLP(
  (net): Sequential(
    (0): Linear(in_features=40, out_features=80, bias=True)
    (1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=80, out_features=160, bias=True)
    (4): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=160, out_features=160, bias=True)
    (7): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=160, out_features=640, bias=True)
    (10): BatchNorm1d(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=640, out_features=20, bias=True)
    (13): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Linear(in_features=20, out_features=10, bias=True)
    (16): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=Tru

In [39]:
n_epochs = 20
Train_loss = []
Test_loss = []
Test_acc = []
import time
for i in range(n_epochs):
    train_loss = train_epoch(model,train_dataloader, criterion, optimizer)
    test_loss, test_acc = test_model(model, test_dataloader, criterion)
    Train_loss.append(train_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)
    print('='*20)

Training Loss:  0.7245642651211132 Time:  0.22744512557983398 s
Testing Loss:  0.8759328961372376
Testing Accuracy:  48.49498327759198 %
Testing Recall:  89.0842490842491 %
Training Loss:  0.47149265625260095 Time:  0.3040003776550293 s
Testing Loss:  0.5800063490867615
Testing Accuracy:  76.92307692307693 %
Testing Recall:  52.761904761904766 %
Training Loss:  0.4361747584559701 Time:  0.32303738594055176 s
Testing Loss:  0.4899646699428558
Testing Accuracy:  80.2675585284281 %
Testing Recall:  31.382539029597854 %
Training Loss:  0.42177595604549756 Time:  0.3685920238494873 s
Testing Loss:  0.4423814594745636
Testing Accuracy:  80.93645484949833 %
Testing Recall:  29.00754147812971 %
Training Loss:  0.4371494027701291 Time:  0.34351491928100586 s
Testing Loss:  0.44339950680732726
Testing Accuracy:  80.93645484949833 %
Testing Recall:  33.62895927601811 %
Training Loss:  0.38474084301428363 Time:  0.3500382900238037 s
Testing Loss:  0.4400504469871521
Testing Accuracy:  82.608695652

In [42]:
n_epochs = 20
Train_loss = []
Test_loss = []
Test_acc = []
import time
for i in range(n_epochs):
    train_loss = train_epoch(model,train_dataloader_Y, criterion, optimizer)
    test_loss, test_acc = test_model(model, test_dataloader, criterion)
    Train_loss.append(train_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)
    print('='*20)

Training Loss:  0.1277914590069226 Time:  0.24417448043823242 s
Testing Loss:  0.7860221028327942
Testing Accuracy:  66.22073578595318 %
Testing Recall:  80.67032967032966 %
Training Loss:  0.12145556030528885 Time:  0.3705925941467285 s
Testing Loss:  0.7429047703742981
Testing Accuracy:  69.23076923076923 %
Testing Recall:  76.6307418072124 %
Training Loss:  0.1712374304022108 Time:  0.3915591239929199 s
Testing Loss:  0.6885405063629151
Testing Accuracy:  71.23745819397993 %
Testing Recall:  76.98368298368298 %
Training Loss:  0.13311656404818809 Time:  0.4265122413635254 s
Testing Loss:  0.9154417514801025
Testing Accuracy:  58.19397993311036 %
Testing Recall:  81.33433233433234 %
Training Loss:  0.10788762090461594 Time:  0.38900279998779297 s
Testing Loss:  0.7549282550811768
Testing Accuracy:  67.89297658862876 %
Testing Recall:  81.13347763347763 %
Training Loss:  0.11785744662795748 Time:  0.44199490547180176 s
Testing Loss:  0.7777569770812989
Testing Accuracy:  67.8929765886